# Model Comparison: HSANet vs ViT, Swin, ResNet, VGG

This notebook trains and evaluates multiple architectures for comparison.

**Run on Kaggle with GPU enabled.**

In [ ]:
!pip install timm -q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import timm
import numpy as np
from PIL import Image
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = self.root_dir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.jpg'):
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
                for img_path in class_dir.glob('*.png'):
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
        
        print(f"Loaded {len(self.samples)} images from {root_dir}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Update paths for Kaggle
TRAIN_DIR = '/kaggle/input/brain-tumor-mri-dataset/Training'
TEST_DIR = '/kaggle/input/brain-tumor-mri-dataset/Testing'

train_dataset = BrainTumorDataset(TRAIN_DIR, train_transform)
test_dataset = BrainTumorDataset(TEST_DIR, test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Train: {len(train_dataset)}, Test: {len(test_dataset)}")

In [ ]:
def train_and_evaluate(model_name, model, train_loader, test_loader, epochs=15):
    """Train a model and return metrics"""
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        scheduler.step()
        
        model.eval()
        all_preds, all_labels, all_probs = [], [], []
        
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)
                preds = outputs.argmax(dim=1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.numpy())
                all_probs.extend(probs.cpu().numpy())
        
        acc = accuracy_score(all_labels, all_preds)
        print(f"  Epoch {epoch+1}: Loss={train_loss/len(train_loader):.4f}, Acc={acc:.4f}")
        
        if acc > best_acc:
            best_acc = acc
            best_preds = all_preds
            best_probs = all_probs
            best_labels = all_labels
    
    training_time = time.time() - start_time
    
    f1 = f1_score(best_labels, best_preds, average='macro')
    try:
        auc = roc_auc_score(best_labels, best_probs, multi_class='ovr', average='macro')
    except:
        auc = 0.0
    
    # Inference time
    model.eval()
    dummy = torch.randn(1, 3, 224, 224).to(device)
    for _ in range(10):  # warmup
        _ = model(dummy)
    
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(100):
        _ = model(dummy)
    torch.cuda.synchronize()
    inference_time = (time.time() - start) / 100 * 1000
    
    params = sum(p.numel() for p in model.parameters()) / 1e6
    
    results = {
        'model': model_name,
        'accuracy': best_acc * 100,
        'f1': f1 * 100,
        'auc': auc,
        'params_M': params,
        'inference_ms': inference_time,
    }
    
    print(f"\n{model_name} Results: Acc={results['accuracy']:.2f}%, F1={results['f1']:.2f}%, Params={params:.2f}M")
    return results

In [ ]:
# Train ViT-B/16
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=4)
vit_results = train_and_evaluate('ViT-B/16', vit_model, train_loader, test_loader, epochs=15)

In [ ]:
# Train Swin-Tiny
swin_model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=4)
swin_results = train_and_evaluate('Swin-Tiny', swin_model, train_loader, test_loader, epochs=15)

In [ ]:
# Train ResNet-50
resnet_model = timm.create_model('resnet50', pretrained=True, num_classes=4)
resnet_results = train_and_evaluate('ResNet-50', resnet_model, train_loader, test_loader, epochs=15)

In [ ]:
# Train VGG-16
vgg_model = timm.create_model('vgg16', pretrained=True, num_classes=4)
vgg_results = train_and_evaluate('VGG-16', vgg_model, train_loader, test_loader, epochs=15)

In [ ]:
# Summary
all_results = [vit_results, swin_results, resnet_results, vgg_results]
df = pd.DataFrame(all_results)
df = df[['model', 'params_M', 'accuracy', 'f1', 'auc', 'inference_ms']]
df.columns = ['Method', 'Params (M)', 'Accuracy (%)', 'F1 (%)', 'AUC-ROC', 'Inference (ms)']

print("\n" + "="*80)
print("FINAL RESULTS - Use these values in your paper!")
print("="*80)
print(df.to_string(index=False))
print("="*80)

df.to_csv('comparison_results.csv', index=False)
print("\nResults saved to comparison_results.csv")